In this notebook i will use **TMDB 5000 Movie Dataset**.

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool
import json  # will use for converting json data to str

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# First load data
data = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')


In [ ]:
# look data inside
data.head()


In [ ]:
data.info()

The dataset has the following features:-

* budget - The budget in which the movie was made.
* genres - The genre of the movie, Action, Comedy ,Thriller etc.
* homepage - A link to the homepage of the movie.
* keywords - The keywords or tags related to the movie.
* original_language - The language in which the movie was made.
* original_title - The title of the movie before translation or adaptation.
* overview - A brief description of the movie.
* popularity - A numeric quantity specifying the movie popularity.
* production_companies - The production house of the movie.
* production_countries - The country in which it was produced.
* release_date - The date on which it was released.
* revenue - The worldwide revenue generated by the movie.
* runtime - The running time of the movie in minutes.
* status - "Released" or "Rumored".
* tagline - Movie's tagline.
* title - Title of the movie.
* vote_average - average ratings the movie recieved.
* vote_count - the count of votes recieved.

# 1-)Highest budget movies


In [ ]:
highestBudgetMovies = data.nlargest(10, 'budget')  # we keep 10 movies which has highest budget
plt.subplots(figsize=(12, 10))
ax = pd.Series(highestBudgetMovies.budget).sort_values(ascending=True).plot.barh(width=0.9,
                                                                                 color=sns.color_palette('rocket', 10))
for i, v in enumerate(highestBudgetMovies.title):
    ax.text(.8, i, v, fontsize=12, color='white', weight='bold')
plt.title('Highest budget movies')
plt.xlabel("Total budget in $")
ax.set_yticklabels([])  # hide movie's id
plt.show()


# 2-) Relation between **popularity** and **budget**

In [ ]:
sns.relplot(kind='scatter', x='budget', y='popularity', data=data)
plt.show()


* And you can see that high popularity doesnt come with high budget :)

# 3-) Top 10 Genres

In [ ]:
# lets look genres
print(data.genres)


*  We have to find unique genres values
*  As we saw our  genres in json format, first we will convert to strings.

In [ ]:
# JSON to STRING
# json.loads; used for parse valid JSON String into Python dictionary.
# range; allows to generate a series of numbers within a given range. Ex; 0,1,2...10
# append; adds a single item to the existing list
data['genres'] = data['genres'].apply(json.loads)  # with apply function each of data.genres will become string
for index, i in zip(data.index, data['genres']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))  # 'name' key contains name of the genre

    data.loc[index, 'genres'] = str(list1)


In [ ]:
# STRING to LIST
# The strip; removes characters from both left and right based on the arg.
data['genres'] = data['genres'].str.strip('[]').str.replace(' ', '').str.replace("'", '')
data['genres'] = data['genres'].str.split(',')


What we did here ???

* We deleted space with *str.replace('  ' , '')*
* We replaced "'" character with space with *str.replace(" ' " , '')*

['Action', 'Adventure', 'Crime'] => [Adventure, Fantasy, Action]

In [ ]:
plt.subplots(figsize=(12, 10))
list1 = []
# extend(); takes a single argument (a list) and adds it to the end.
# value_counts(); return a series containing counts of unique values.
for i in data.genres:
    list1.extend(i)  # we add all genres to one list
ax = pd.Series(list1).value_counts()[:10].sort_values(ascending=True).plot.barh(width=0.9,
                                                                                color=sns.color_palette('spring', 10))

# We take each value of unique genres and indexing them like (0:2297),(1:1722) etc and write them to bar.
for i, v in enumerate(pd.Series(list1).value_counts()[:10].sort_values(ascending=True).values):
    ax.text(.8, i, v, fontsize=12, color='white', weight='bold')
plt.title('Top 10 Genres')
plt.show()


# 4-)Good movie or not ?

For this we will use list comprehension and **vote_average** column.

In [ ]:
data['nice'] = ['good' if each < data.vote_average.mean() else 'not' for each in data.vote_average]
# We add new column to our dataset which has 2 unique value
data.nice.unique()


Lets see the rate !
We will show this with Pie Chart.


In [ ]:
#  Pie chart, where the slices will be ordered and plotted counter-clockwise.
labels = 'Good', 'Not Good'
sizes = data['nice'].value_counts()  # sizes have number for 'not' and 'good'
explode = (0.05, 0)  # first part of pie will be explode
colors = ["lightpink", "yellowgreen"]
plt.pie(sizes, explode=explode, colors=colors, autopct='%1.1f%%', shadow=True)  # autopct; wrote rates on pies
plt.title('Good movie or not?')
plt.legend(labels, loc='upper right')
plt.show()


# 5-) Short movie or not ?

In [ ]:
data['time'] = ['short' if each < data.runtime.mean() else 'not' for each in data.runtime]
# We add new column to our dataset which has 2 unique value
data.time.unique()


In [ ]:
labels = 'Short', 'Not Short'
sizes = data['time'].value_counts()  # sizes have number for 'not' and 'short'
explode = (0.05, 0)  # first part of pie will be explode
colors = ["bisque", "aqua"]
plt.pie(sizes, explode=explode, colors=colors, autopct='%1.1f%%', shadow=True)  # autopct; wrote rates on pies
plt.title('Short movie or not?')
plt.legend(labels, loc='upper right')
plt.show()


# 6-)  Relation between vote average and runtime

In [ ]:
sns.relplot(kind='line', x='vote_average', y='runtime', data=data)
plt.show()
